# Creando un Entorno

Fuente: https://www.youtube.com/watch?v=Mut_u40Sqz4

A partir del minuto 2:30:00

Vamos a enumerar los pasos para hacer un trabajo de RL:
- Importar dependencia
- Construir el entorno
- Testear el entorno
- Entrenar el modelo
- Guardar/Cargar el modelo
- Evaluar el modelo

## Importar Dependencias

In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
# En este ejecicio vamos a crear nosotros mismos un entorno
# Estas 2 últimas dependencias es lo que necesitaremos para crearlo

import numpy as np
import random
import os
# Esto es más que obvio lo que hace

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
# Todo esto está en lo de apuntes

## Tipos de espacios

In [2]:
Discrete(3).sample()
# Los espacios de tipo discreto generan resultados discretos
# El parametro es numero de posibles acciones que se pueden hacer
# Aquí dará valores 0,1 ó 2, cada uno debe corresponder a una accion diferente

1

In [3]:
Box(0,1,shape=(3,3)).sample()
# Con Box generamos acciones continuas
# Le pasamos el valor mínimo, el máximo, y en shape del resultado
# Aquí como la forma es 3,3, generará un array 3x3 de valores entre 0 y 1
Box(0,1,shape=(3,)).sample()# Genera array de 3x1

array([0.07350674, 0.24759133, 0.78674775], dtype=float32)

In [4]:
MultiBinary(4).sample()
# Generar muchos valores todos binarios
# El numero que le pasamos como parametro es el número de muestras que genera
# Todo es binario, o 0 o 1

array([0, 1, 1, 1], dtype=int8)

In [5]:
MultiDiscrete([5,2,2]).sample()
# La pasas una lista de valores, y genera un valor discreto para cada valor de la lista
# Aquí le hemos pasado 3 valores, así que generará 3 numeros
# El primero será de 0 a 4, el segundo y tercero de 0 a 1

array([0, 0, 1], dtype=int64)

In [6]:
Tuple((Discrete(5), Box(0,1,shape=(3,2)), MultiDiscrete([3,3,8]))).sample()
# Podemos combinar espacios
# Se usa para eso Tuple, así genero 1 valor discreto y 3x2 continuos
# Destacar que hay que poner los parentesis bien, hay uno dentro que junta Discrete y Box 

(2,
 array([[0.6497505 , 0.20930882],
        [0.3030709 , 0.7328464 ],
        [0.85771656, 0.274109  ]], dtype=float32),
 array([1, 0, 0], dtype=int64))

In [7]:
Dict({"height":Discrete(2), "Speed":Box(0,100, shape=(1,))}).sample()
# Aquí tenemos que pasar un diccionario
# Podemos juntar diferentes tipos de espacios, y a cada uno ponerle una etiqueta

OrderedDict([('Speed', array([50.36152], dtype=float32)), ('height', 1)])

## Construir un entorno

El problema en este caso será:
- Construir un agente que se nos dé la mejor ducha posible
- Temperatura aleatoria
- Entre 37 y 39 grados

Nuestro agente no conoce lo que nosotrso queremos, debe verlo por las recompensas que le damos

In [8]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        # Las 3 acciones que tomaremos serán subir la temperatura, bajarla o no cambiarla
        self.observation_space = Box(low = np.array([0]), high = np.array([100]))
        # El espacio de observaciones será un único valor entre 0 y 100
        self.state = 38 + random.randint(-3,3)
        # Esto será el estado inicial, un numero entero entre 35 y 41
        # Este es el que definirá cómo nos encontramos, el estado en el que estamos
        # Queremos que esté entre 37 y 39
        self.shower_length = 60
        # La longitud de un episodio
    
    def step(self, action):
        # Aquí decrementaremos la longitud del episodio en 1
        # Ejecutaremos una acción y daremos una recompensa

        self.state += action - 1
        # Las aciones son subir, mantener o bajar la temperatura
        # Así que como las acciones son numeros entre 0 y 2, el estado cambiará según el valor
        # Si el valor es 0 restará 1, si el 1 se mantendrá y 2 sumará 1
        
        self.shower_length -= 1
        # Bajamos el tiempo de la ducha
        # Cada vez que hacemos una accion bajamos el tiempo

        # Calculamos la recompensa
        # Es lo más importante
        if self.state >= 37 and self.state <= 39:
            reward = 1
            # Si la ducha está en el intervalo de temperatura deseado
            # Recompensa positiva
        else:
            # En otro caso, recompensa negativa
            reward = -1
            
        if self.shower_length <= 0:
            # Mientras no hayamos hecho 60 pasos
            # El episodio no ha acabado
            done = True
        else:
            done = False

        # Si quieres pasar información adicional, puedes
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        # Aquí no haremos nada, pero podriamos hacer si queremos
        pass
    def reset(self):
        # Aquí es si queremos resetear el entorno
        # Que vuelva a su estado inicial

        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

## Probar entorno

In [9]:
env = ShowerEnv()

C:\Users\jpeir\anaconda3\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [10]:
env.observation_space.sample()

array([50.557068], dtype=float32)

In [11]:
env.action_space.sample()

1

In [12]:
env.reset()

array([41.])

In [13]:
# Esta es muy buena manera de probar un entorno
# Que se basa en probar movimientos aleatorios a ver qué pasa
# El código viene copiado de los apuntes de antes

episodes = 5 # Probamos 5 veces
for ep in range(episodes):
    state = env.reset()
    # Reinicia el entorno
    done = False
    # Estado inicial
    score = 0
    # Recompensa/Puntuación
    
    while not done:
        env.render()
        # Esto no hace falta, no tenemos render
        
        action = env.action_space.sample()
        # Realiza una acción aleatoria
        
        n_state, reward, done, info = env.step(action)
        # Ejecutamos la accion elegida con step
        
        score += reward
    print("Episode:{} Score:{}".format(ep, score))
#env.close()

Episode:0 Score:-8
Episode:1 Score:-26
Episode:2 Score:20
Episode:3 Score:-50
Episode:4 Score:-44


## Entrenar modelo

In [14]:
log_path = os.path.join("Entrenamiento", "Logs")
# Pasamos el directorio para que se almacenen registros

model = PPO("MlpPolicy", env, verbose = 1, tensorboard_log = log_path)
# Creamos el modelo
# utomaticamente, como no hemos hecho nada se almacena como un DummyVecEnv
# Eso quiere decir que no se vectoriza el entorno
# Pero bueno, no pasa nada, este ejemplo es sencillo
# Usamos la politica de perceptron multicapa, solo tenemos datos tabulares, no imagenes
# Pasamos el entorno, que muestre resultados y el camino al directorio

C:\Users\jpeir\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [15]:
model.learn(total_timesteps=999999, tb_log_name = "PPO_999999")
# Se entrena más o menos rápido
# El modelo es super sencillo
# El parametro ep_rew_mean muestra la recompensa media

Logging to Entrenamiento\Logs\PPO_999999_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -30.5    |
| time/              |          |
|    fps             | 815      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -29.1       |
| time/                   |             |
|    fps                  | 664         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010424828 |
|    clip_fraction        | 0.0473      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -

### Muy importante!!!!
Podemos ver que se sobreentrena, a partir de las iteraciones 500000

Hasta la 350000 va espectacular, a partir de ahí no para de empeorar

Y el resultado al final es nefasto. Para tenerlo en cuenta...

Conclusión: No sobreentrenar

## Guardar modelo

In [16]:
shower_path = os.path.join("Entrenamiento", "Saved Models", "Modelo_Ducha_PPO")
# Directorio donde se guardará el modelo

model.save(shower_path)
# Guardamos el modelo entrenado

In [17]:
# Como hemos guardado el modelo en una carpeta podemos eliminarlo y volverlo a cargar
del model

In [18]:
try:
    model
except NameError as error:
    print("No existe la variable")
    print(error)

No existe la variable
name 'model' is not defined


In [19]:
# Podemos volver a cargar el modelo que teniamos
model = PPO.load(shower_path, env)
# Le tenemos que pasar el entorno, eso sí

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\jpeir\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


## Evaluamos el modelo

In [51]:
evaluate_policy(model, env, n_eval_episodes=1000, render = True)
# Y el resultado de esto es la media y la desviación típica de los scores
# Podemos valorar que ha mejorado un poquito (no demasiado)

(-10.32, 59.10581697261277)